# Processamento de dados Institucional

## 1. Imports

In [35]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from typing import List, Dict, Tuple

# Perguntas
df_dados_perguntas_raw = pd.read_csv('../raw/dados_institucional_perguntas.csv')
df_dados_info_perguntas_raw = pd.read_csv('../raw/dados_institucional_info_perguntas.csv')
df_dados_info_questionarios_raw = pd.read_csv('../raw/dados_institucional_info_questionarios.csv')
# Unidades
df_dados_unidades_raw = pd.read_csv('../raw/dados_institucional_unidades.csv')

# Pesquisa
df_dados_pesquisa_raw = pd.read_csv('../raw/dados_institucional_pesquisa.csv')

## 2. Funções para descrição de dataframes

In [36]:
def get_possible_results(df: pd.DataFrame) -> Dict:
    ''' 
    Return:
    - data: Dictionary where the keys are the columns values and values are a list containing all possible values in a column. Order of appearance is preserved by .unique().
    
    '''
    cols = df.columns
    data = dict()

    for col in cols:
        data[col] = df[col].unique().tolist()

    return data

In [37]:
def remove_unnamed_columns(df: pd.DataFrame) -> pd.DataFrame:
    ''' 
    Laziness in manually processing data before transforming it into dataframes.

    Remove all the 'Unnamed' columns in a dataframe.
    '''
    return df.loc[:,~df.columns.str.contains('^Unnamed')]

In [46]:
def count_occurrences_by_column(df: pd.DataFrame,columns: List|None = None) -> Dict:
    columns_to_process = columns if columns is not None else df.columns

    occurrences = {}

    for col in columns_to_process:
        occurrences[col] = df[col].value_counts()

    return occurrences

### 2.1 Testes

In [39]:
print(get_possible_results(remove_unnamed_columns(df_dados_info_perguntas_raw)))

{'Tipo_Perg': ['MsãoPDI', 'RsSc', 'PolPes', 'OrgGesInst', 'SustFin', 'CHC'], 'Grupo_de_Perguntas': ['Missão e Plano de Desenvolvimento Institucional', 'Responsabilidade Social', 'Políticas de Pessoal', 'Organização e Gestão da Instituição', 'Sustentabilidade Financeira', 'Complexo do Hospital de Clínicas (CHC-UFPR/Ebserh)']}


In [40]:
remove_unnamed_columns(df_dados_info_questionarios_raw)


,ID_QUESTIONARIO,QUESTIONARIO
0,644,Missão e Plano de Desenvolvimento Institucional
1,645,Responsabilidade Social
2,646,Políticas de Pessoal
3,647,Organização e Gestão da Instituição
4,648,Sustentabilidade Financeira
5,649,Complexo do Hospital de Clínicas (CHC-UFPR/Ebs...


In [ ]:
ocorrencias = count_occurrences_by_column(df_dados_pesquisa_raw)

for coluna, contagem in ocorrencias.items():
    print(f"--- Frequência: {coluna} ---")
    print(contagem.to_string())
    print("\n")

## 3. Processamento dos dados

Nesta parte, traremos informações relevantes para uma única tabelas (em torno da tabela `df_dados_pesquisa`).

Aqui veremos a relação de cada tabela com a nossa principal.

### 3.1 Info_Perguntas e Info_Questionários

---

In [52]:
df_dados_info_perguntas_raw = remove_unnamed_columns(df_dados_info_perguntas_raw)
df_dados_info_perguntas_raw

,Tipo_Perg,Grupo_de_Perguntas
0,MsãoPDI,Missão e Plano de Desenvolvimento Institucional
1,RsSc,Responsabilidade Social
2,PolPes,Políticas de Pessoal
3,OrgGesInst,Organização e Gestão da Instituição
4,SustFin,Sustentabilidade Financeira
5,CHC,Complexo do Hospital de Clínicas (CHC-UFPR/Ebs...


Talvez seja uma boa ideia trazer o `Tipo_Perg` para a tabela principal, pois tornaria mais fácil a visualização da distribuição de respostas na tabela principal.

---

In [53]:
df_dados_info_questionarios_raw = remove_unnamed_columns(df_dados_info_questionarios_raw)
df_dados_info_questionarios_raw

,ID_QUESTIONARIO,QUESTIONARIO
0,644,Missão e Plano de Desenvolvimento Institucional
1,645,Responsabilidade Social
2,646,Políticas de Pessoal
3,647,Organização e Gestão da Instituição
4,648,Sustentabilidade Financeira
5,649,Complexo do Hospital de Clínicas (CHC-UFPR/Ebs...


Ambas as informações já estão na tabela principal.

### 3.2 Unidades

In [54]:
df_dados_unidades_raw

,SIGLA_LOTACAO,UNIDADE GESTORA,LOTACAO
0,AC/DEARTES,"Setor de Artes, Comunicação e Design",Departamento de Artes
1,AC/DECOM,"Setor de Artes, Comunicação e Design",Departamento de Comunicação Social
2,AG/CCEIM,Setor de Ciências Agrárias,Coordenação do Curso de Engenharia Industrial ...
3,AG/DERE,Setor de Ciências Agrárias,Departamento de Economia Rural e Extensão
4,AG/DETF,Setor de Ciências Agrárias,Departamento de Engenharia e Tecnologia Florestal
...,...,...,...
310,UNIMULTI/SMFR,Hospital de Clínicas,Serviço Medicina Física e Reabilitação
311,UNP/NEUROMUSC,Hospital de Clínicas,Serviço de Doenças Neuromusculares
312,UPAT/SCBM,Hospital de Clínicas,Seção de Cadastro de Bens Móveis
313,UUEA/CTSIADULTO,Hospital de Clínicas,Centro de Terapia Semi Intensiva Adulto


A única coluna que não aparece na tabela principal é a `UNIDADE GESTORA`

---

### 3.3 Perguntas

Colunas de `df_dados_perguntas_raw`:
* `ID_PERGUNTA` - Está na tabela principal; Autoexplicativo
* `ID_QUESTIONARIO` - Está na tabela principal; Autoexplicativo
* `TITULO` - Está na tabela principal; Equivalente à `QUESTIONARIO`
* `CL_PERGUNTA` - Não está na tabela principal;Todas as observações são `Institucional`
* `Ordem` - Não está na tabela principal; Ordem das perguntas feitas em cada questionário
* `Coluna1` - Não está na tabela principal; Equivalente à `Tipo_Perg` da tabela `df_dados_info_perguntas_raw`
* `PERGUNTA` - Está na tabela principal; Autoexplicativo